# Create point defects in a bulk material

Create a vacancy, add substitution or interstitial atom to provided Material.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook. 
1. Set notebook parameters in cell 1.1. below (or use the default values) -- to create multiple defects, fill `DEFECT_CONFIGS` with a list of dictionaries. 
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 

## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defects parameters 
Defect Configuration parameters are described in [Defect Configuration](https://github.com/Exabyte-io/made/blob/8196b759242551c77d1791bf5bd2f4150763cfef/src/py/mat3ra/made/tools/build/defect/configuration.py#L102).

In [ ]:
DEFECT_TYPE = "substitution"  # (e.g. "vacancy", "substitution", "interstitial")
SITE_ID = None  # Site index of the defect
COORDINATE = None  # Position of the defect in crystal coordinates
APPROXIMATE_COORDINATE = None  # Approximate coordinates of the defect in crystal coordinates
CHEMICAL_ELEMENT = "C"  # Element to be placed at the site (ignored for vacancy)

SUPERCELL_MATRIX = [[3, 0, 0], [0, 3, 0], [0, 0, 3]]

# List of dictionaries with defect parameters
DEFECT_CONFIGS = [
    {
        "defect_type": DEFECT_TYPE,
        "approximate_coordinate": [0.0, 0.0, 0.0],
        "chemical_element": "C",
        # "site_id": 0,
        # "coordinate": None,
    },
    # Uncomment the following to create multiple defects
    # {
    #     "defect_type": "vacancy",
    #     "site_id": 0,
    # },
]

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("create_point_defect.ipynb")

### 1.3. Get input materials
Materials are loaded with `get_materials()`.

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

### 1.4. Create and preview Supercell

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.supercell import create_supercell

unit_cell = materials[0]
supercell = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create the Defect
### 2.1. Initialize Configuration and Builder parameters

In [ ]:
from mat3ra.made.tools.build.defect import PointDefectConfiguration
from mat3ra.made.tools.build.defect.builders import PointDefectBuilderParameters

defect_configurations = [PointDefectConfiguration.from_dict(supercell, defect) for defect in DEFECT_CONFIGS]

defect_builder_parameters = PointDefectBuilderParameters(center_defect=False)

### 2.2. Create the defects

In [ ]:
from mat3ra.made.tools.build.defect import create_defects

material_with_defect = create_defects(builder_parameters=defect_builder_parameters, configurations=defect_configurations)

## 3. Visualize Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}],
          rotation="-90x")
visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials([material_with_defect])